In [3]:
import pandas as pd
import numpy as np

In [4]:
train =pd.read_csv("D:/Hackethon/recommend/train_submissions.csv")

In [5]:
# visualizing the dataset
train.head(3)

,user_id,problem_id,attempts_range
0,user_232,prob_6507,1
1,user_3568,prob_2994,3
2,user_1600,prob_5071,1


In [6]:
# we will use matrix factorization to solve this problem, intially we will not consider the additiona;l features such as user and problem details
rc_Df = train.pivot(index="user_id",columns="problem_id",values="attempts_range").fillna(0)

In [7]:
# The basic idea is that we will create two more matrix and try to approximate the original user-item-rating matrix using these.
# the difference will be that these new matixes will of size UK and IK
# where U- number of users, I- number of items and K- number of latent features we wish to find(for general purpose it varies between 20 to 100)

2.0

In [119]:
# function for matrix factorization with gradient descent with regularzation
def matrix_factorization(R, P, Q, K, steps=100, alpha=0.0002, beta=0.02):
    
    Q = Q.T

    for step in range(steps):

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):

                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])

                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = np.dot(P,Q)

        e = 0

        for i in range(len(R)):

            for j in range(len(R[i])):

                if R[i][j] > 0:

                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):

                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))

        if e < 0.001:

            break

    return P, Q.T


In [120]:
R = np.array(rc_Df)
N = len(R)

M = len(R[0])

K = 20
P = np.random.rand(N,K)

Q = np.random.rand(M,K)

 

nP, nQ = matrix_factorization(R, P, Q, K)

nR = np.dot(nP, nQ.T)


In [121]:
newdf1 = pd.DataFrame(nR,columns=rc_Df.columns)

In [122]:
import math
#converting the float to smallest possible integer
newdf1 =newdf1.applymap(lambda x:math.floor(x))


In [134]:
#Since minimum value is 1 and max is 6, converting these values to the nearest value
newdf1 =newdf1.applymap(lambda x:1 if x<1 else x)
newdf1 =newdf1.applymap(lambda x:6 if x>6 else x)


In [135]:
updatedDF = pd.concat([userdf,newdf1],axis=1)

In [136]:
updatedDF.head(1)

,userID,prob_1,prob_10,prob_100,prob_1000,prob_1001,prob_1002,prob_1003,prob_1004,prob_1005,...,prob_990,prob_991,prob_992,prob_993,prob_994,prob_995,prob_996,prob_997,prob_998,prob_999
0,user_1,2,2,2,1,1,1,2,2,1,...,2,1,2,1,2,2,1,1,2,1


In [143]:
test =pd.read_csv("D:/Hackethon/recommend/test.csv")

In [138]:
def getAttemptsRating(tarDF,df,var1,var2):
    rateList =[]
    for i,r in df.iterrows():
        try:
            user_id= r["user_id"]
            Rating= r["problem_id"]
            rateval = tarDF.loc[tarDF.userID == user_id,Rating].values[0]
            rateList.append(rateval)
        except:
            rateList.append(6)
    return rateList
        
    

In [139]:
rates = getAttemptsRating(updatedDF,test,"user_id","problem_id")

In [144]:
test["attempts_range"]=rates

In [145]:
test.drop(["user_id","problem_id"],axis=1,inplace=True)

In [147]:
test.to_csv("D:/Hackethon/recommend/submission_new_100_1.csv")